In [28]:
import scipy
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

### TEST 1 Доверительные интервалы

1.Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [23]:
alpha = 1-0.997
norm.ppf(alpha/2)

-2.9677379253417833

2.В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.
За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.
Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
p_placebo = 189./11034
p_aspirin = 104./11037

a5 = p_placebo-p_aspirin
print a5

0.007706023976


2.Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [4]:
na = 11037
na_i = 104
data_a = np.concatenate([np.ones((na_i)), np.zeros((na-na_i))])

In [26]:
a6 = _tconfint_generic(data_a.mean(), data_a.std()/np.sqrt(len(data_a)), len(data_a) - 1, 0.05, 'two-sided')
print a6

(0.0076202249371126393, 0.01122547588738677)


3.Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p1−p, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. 
Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [14]:
odds_p = p_placebo/(1-p_placebo)
odds_a = p_aspirin/(1-p_aspirin)
odds_p/odds_a

1.8320539419087138

4.Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас: составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
установите random seed=0;
сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [13]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [15]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [16]:
def get_odds(sample):
    return sum(sample)/(len(sample)-sum(sample))

In [17]:
np.random.seed(0)
samples_a = get_bootstrap_samples(data_a,1000)
samples_p = get_bootstrap_samples(data_p,1000)

In [18]:
odds_p = map(get_odds, samples_p)
odds_a = map(get_odds, samples_a)
odds = np.array(odds_p)/np.array(odds_a)

In [19]:
a8 = _tconfint_generic(odds.mean(), odds.std()/np.sqrt(len(odds)), len(odds) - 1, 0.05, 'two-sided')
print a8

(1.833798702578872, 1.862909063196182)


### TEST 2 Проверка гипотез

1.По данным опроса, 75% работников ресторанов утверждают, что испытывают на работе существенный стресс, оказывающий негативное влияние на их личную жизнь. Крупная ресторанная сеть опрашивает 100 своих работников, чтобы выяснить, отличается ли уровень стресса работников в их ресторанах от среднего. 67 из 100 работников отметили высокий уровень стресса.

Посчитайте достигаемый уровень значимости, округлите ответ до четырёх знаков после десятичной точки.

In [24]:
print "%.6f" % stats.binom_test(22, 50, 0.75, alternative = 'two-sided')

0.000003


2.The Wage Tract — заповедник в округе Тома, Джорджия, США, деревья в котором не затронуты деятельностью человека со времён первых поселенцев. Для участка заповедника размером 200х200 м имеется информация о координатах сосен (sn — координата в направлении север-юг, we — в направлении запад-восток, обе от 0 до 200).

Проверим, можно ли пространственное распределение сосен считать равномерным, или они растут кластерами.

Загрузите данные, поделите участок на 5х5 одинаковых квадратов размера 40x40 м, посчитайте количество сосен в каждом квадрате (чтобы получить такой же результат, как у нас, используйте функцию scipy.stats.binned_statistic_2d).

Если сосны действительно растут равномерно, какое среднее ожидаемое количество сосен в каждом квадрате? В правильном ответе два знака после десятичной точки.

In [29]:
trees=pd.read_csv('pines.txt',delimiter='\t')


In [30]:
trees.head()

,sn,we
0,200.0,8.8
1,199.3,10.0
2,193.6,22.4
3,167.7,35.6
4,183.9,45.4


In [31]:
ret = stats.binned_statistic_2d(trees.we, trees.sn,None,'count', bins=[5,5])
observed = ret.statistic

In [32]:
m = observed.mean()
print m
print observed

23.36
[[ 18.  14.  12.   8.   7.]
 [ 26.  12.  24.  19.   8.]
 [ 29.  22.  15.  31.  37.]
 [ 25.  34.  50.  50.  27.]
 [ 20.  25.  37.   7.  27.]]


3.Чтобы сравнить распределение сосен с равномерным, посчитайте значение статистики хи-квадрат для полученных 5х5 квадратов. Округлите ответ до двух знаков после десятичной точки.

In [33]:
expected = m*np.ones((5,5))
expected

array([[ 23.36,  23.36,  23.36,  23.36,  23.36],
       [ 23.36,  23.36,  23.36,  23.36,  23.36],
       [ 23.36,  23.36,  23.36,  23.36,  23.36],
       [ 23.36,  23.36,  23.36,  23.36,  23.36],
       [ 23.36,  23.36,  23.36,  23.36,  23.36]])

In [34]:
stats.chisquare(observed.reshape(25), expected.reshape(25), ddof = 0)


Power_divergenceResult(statistic=150.58904109589042, pvalue=2.5746697749672791e-20)